In [1]:
%run imports.py

In [2]:
# Import relevant tables
job_post = pd.read_csv('Data/job_postings.csv')
job_skills = pd.read_csv('Data/job_skills.csv')
skills_map = pd.read_csv('Data/skills.csv')
job_industries = pd.read_csv('Data/job_industries.csv')
industries_map = pd.read_csv('Data/industries.csv')

In [3]:
# Add skills and industries to job_post DF 
job_skills = job_skills.merge(skills_map)[['job_id','skill_name']]
job_industries = job_industries.merge(industries_map)[['job_id','industry_name']]
jobs = job_post.merge(job_skills).merge(job_industries)

In [4]:
# Drop rows where all of the salary fields are Null and rows where the work type isn't full time
salary_cols = ['max_salary','med_salary','min_salary']
jobs_cleaned = jobs.dropna(subset=salary_cols, how='all')
jobs_cleaned = jobs_cleaned[jobs_cleaned.formatted_work_type == 'Full-time']

In [5]:
# Transform all salaries to be Yearly based on the pay period (Monthly is 12x and Hourly is 2080x)
for column in salary_cols:
    jobs_cleaned.loc[jobs_cleaned['pay_period'] == 'MONTHLY', column] = jobs_cleaned.loc[jobs_cleaned['pay_period'] == 'MONTHLY', column] * 12
    jobs_cleaned.loc[jobs_cleaned['pay_period'] == 'HOURLY', column] = jobs_cleaned.loc[jobs_cleaned['pay_period'] == 'HOURLY', column] * 2080

In [6]:
# Impute missing medians based on median = min+max/2
impute_values = (jobs_cleaned['min_salary'] + jobs_cleaned['max_salary']) / 2

jobs_cleaned.loc[jobs_cleaned['med_salary'].isna(), 'med_salary'] = impute_values

In [7]:
jobs_cleaned.to_csv('jobs_cleaned.csv')